In [21]:
import chess

In [11]:
# load x.npy and y.npy

import numpy as np
import matplotlib.pyplot as plt
#import train test split

# load data
x = np.load('x.npy').reshape(-1, 64)
y = np.load('y.npy')

In [19]:
def get_moves(game):
    moves = []
    for i in game.mainline_moves():
        moves.append(i)
    return moves

In [9]:
print(x[41])
print(y[41])

[  0   0   0   0   5   0  10   0   1   0   0   0   0   1   1   1   0   0
   1   9   0   0   5   0   0   0   0   1   0   0   0   0   0   0   0  -9
   0   0   0   0  -5   0   0   0  -1   0   0   0  -1  -1   0   0   3  -1
  -1  -1   0   0   0   0  -3  -5 -10   0]
3


### The model was trained on colab and saved as a .h5 file

In [ ]:
import numpy as np
import tensorflow as tf
# import train test split
from sklearn.model_selection import train_test_split


# Convert labels to one-hot encoded vectors
y_one_hot = tf.keras.utils.to_categorical(y, num_classes=15)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y_one_hot, test_size=0.2, random_state=42)

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(2048, activation='elu', input_shape=(64,)),
    tf.keras.layers.Dense(2048, activation='elu'),
    tf.keras.layers.Dense(1050, activation='elu'),
    tf.keras.layers.Dense(15, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
hist = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

#plot loss
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Val'], loc='upper right')


In [13]:
#import load_model
from tensorflow.keras.models import load_model
# load model
model = load_model('chessengine.h5')

In [20]:
def check_turn(board_state):
    if board_state.split(' ')[1] == 'w':
        return 'w'
    else:
        return 'b' 

In [ ]:
def get_piece_type(piece):
    # Helper function to get the type of a chess piece (with color information)
    if piece is None:
        return None
    elif piece.color == chess.WHITE:
        if piece.piece_type == chess.PAWN:
            return 'P'
        elif piece.piece_type == chess.KNIGHT:
            return 'N'
        elif piece.piece_type == chess.BISHOP:
            return 'B'
        elif piece.piece_type == chess.ROOK:
            return 'R'
        elif piece.piece_type == chess.QUEEN:
            return 'Q'
        elif piece.piece_type == chess.KING:
            return 'K'
    elif piece.color == chess.BLACK:
        if piece.piece_type == chess.PAWN:
            return 'p'
        elif piece.piece_type == chess.KNIGHT:
            return 'n'
        elif piece.piece_type == chess.BISHOP:
            return 'b'
        elif piece.piece_type == chess.ROOK:
            return 'r'
        elif piece.piece_type == chess.QUEEN:
            return 'q'
        elif piece.piece_type == chess.KING:
            return 'k'



def get_board_matrix(board_state):
    # Initialize an 8x8 matrix to represent the board
    matrix = np.zeros((8, 8), dtype=str)

    # Parse the board_state string to obtain the positions of the pieces on the board
    board = chess.Board(board_state)
    for row in range(8):
        for col in range(8):
            # Get the square index corresponding to the current row and column
            square = chess.square(col, 7 - row)

            # Get the type of the piece occupying the current square (if any)
            piece_type = get_piece_type(board.piece_at(square))

            # Store the piece type in the matrix
            matrix[row][col] = piece_type or '.'

    return matrix

In [ ]:
def label_encode_for_white(board_matrix):

    new_board_matrix = np.zeros((8, 8), dtype=int)
   
    for i in range(len(board_matrix)):
        for j in range(len(board_matrix[i])):
            if board_matrix[i][j] == 'P':
                new_board_matrix[i][j] = 1
            elif board_matrix[i][j] == 'N':
                new_board_matrix[i][j] = 3
            elif board_matrix[i][j] == 'B':
                new_board_matrix[i][j] = 3
            elif board_matrix[i][j] == 'R':
                new_board_matrix[i][j] = 5
            elif board_matrix[i][j] == 'Q':
                new_board_matrix[i][j] = 9
            elif board_matrix[i][j] == 'K':
                new_board_matrix[i][j] = 10
            elif board_matrix[i][j] == '.':
                new_board_matrix[i][j] = 0
            elif board_matrix[i][j] == 'p':
                new_board_matrix[i][j] = -1
            elif board_matrix[i][j] == 'n':
                new_board_matrix[i][j] = -3
            elif board_matrix[i][j] == 'b':
                new_board_matrix[i][j] = -3
            elif board_matrix[i][j] == 'r':
                new_board_matrix[i][j] = -5
            elif board_matrix[i][j] == 'q':
                new_board_matrix[i][j] = -9
            elif board_matrix[i][j] == 'k':
                new_board_matrix[i][j] = -10
    return new_board_matrix


def label_encode_for_black(board_matrix):
    new_board_matrix = np.zeros((8, 8), dtype=int)
    
    for i in range(len(board_matrix)):
        for j in range(len(board_matrix[i])):
            if board_matrix[i][j] == 'P':
                new_board_matrix[i][j] = -1
            elif board_matrix[i][j] == 'N':
                new_board_matrix[i][j] = -3
            elif board_matrix[i][j] == 'B':
                new_board_matrix[i][j] = -3
            elif board_matrix[i][j] == 'R':
                new_board_matrix[i][j] = -5
            elif board_matrix[i][j] == 'Q':
                new_board_matrix[i][j] = -9
            elif board_matrix[i][j] == 'K':
                new_board_matrix[i][j] = -10
            elif board_matrix[i][j] == '.':
                new_board_matrix[i][j] = 0
            elif board_matrix[i][j] == 'p':
                new_board_matrix[i][j] = 1
            elif board_matrix[i][j] == 'n':
                new_board_matrix[i][j] = 3
            elif board_matrix[i][j] == 'b':
                new_board_matrix[i][j] = 3
            elif board_matrix[i][j] == 'r':
                new_board_matrix[i][j] = 5
            elif board_matrix[i][j] == 'q':
                new_board_matrix[i][j] = 9
            elif board_matrix[i][j] == 'k':
                new_board_matrix[i][j] = 10
    return new_board_matrix


In [ ]:
def label_helper(board_state):
    board_matrix = get_board_matrix(board_state)
    if check_turn(board_state) == 'w':
       board_matrix = label_encode_for_black(board_matrix)
    elif check_turn(board_state) == 'b':
       board_matrix = label_encode_for_white(board_matrix)
    return board_matrix
        

In [ ]:
# new label encoding
def label_encode_game_based_on_turn(game):
    board = game.board()
    moves = get_moves(game)
    encoded_moves = []
    for move in moves:
        board.push(move)
        board_state = board.fen()
        # encoded_moves.append(le.fit_transform(get_board_matrix(board.fen()).flatten()))
        encoded_moves.append(label_helper(board_state).flatten())
    return encoded_moves

In [18]:
# play chess
import chess
import chess.svg
import random

board = chess.Board()
board

# play against the model
while not board.is_game_over():

    # get the best move from the model
    x = np.array([label_helper(board.fen()).flatten()])
    y = model.predict(x)[0]
    best_move = np.argmax(y)
    print('best move:', best_move, y[best_move])

    # play the best move
    legal_moves = [move for move in board.legal_moves]
    board.push(legal_moves[best_move])

    # print the board
    print(board)

    # get the human move
    human_move = input('Your move: ')

    # play the human move
    try:
        board.push_san(human_move)
    except:
        print('Illegal move')

    # print the board
    print(board)

1/1 [==============================] - 0s 31ms/step
0
[[0.29778895 0.14795862 0.21430439 0.05152515 0.12829638 0.06799115
  0.01971993 0.03686675 0.01104552 0.00257886 0.00189    0.00111389
  0.0033127  0.00279251 0.01281521]]
